In [1]:
from itertools import islice
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [2]:
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.torch.distributions import NegativeBinomialOutput
from gluonts.torch.modules.loss import NegativeLogLikelihood

from Linear import LinearEstimator

In [3]:
dataset = get_dataset("solar-energy", regenerate=False)

In [4]:
estimator = LinearEstimator(
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    scaling="std",
        
    batch_size=128,
    num_batches_per_epoch=100,
    trainer_kwargs=dict(accelerator="cpu")
)

In [5]:
predictor = estimator.train(
    training_data=dataset.train, 
    cache_data=True, 
    shuffle_buffer_length=1024, 
    validation_data=dataset.test
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\maerzale\.virtualenvs\ConvTS_Mixer-3Rl3B8jo\lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
C:\Users\maerzale\.virtualenvs\ConvTS_Mixer-3Rl3B8jo\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


Sanity Checking: 0it [00:00, ?it/s]

torch.Size([128, 24, 20])
torch.Size([128, 24, 20])


Training: 0it [00:00, ?it/s]

torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])
torch.Size([128, 24, 20])


C:\Users\maerzale\.virtualenvs\ConvTS_Mixer-3Rl3B8jo\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

KeyboardInterrupt



In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,
    predictor=predictor,
)

In [ ]:
forecasts = list(forecast_it)

In [ ]:
tss = list(ts_it)

In [ ]:
evaluator = Evaluator()

In [ ]:
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))

In [ ]:
agg_metrics

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * dataset.metadata.prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()